# Finite Element Method for 2D problem
##### The program below solves a 2D structural problem with concentrated, distributed and gravity forces for displacement field and von-misses stress using Finite Element Method, using a single linear quadratic element with four nodes. (Problem ref: Dr. Clayton pettit's lectures on FEM @ UAlberta)

Scope: To simply solve the problem as stated

In [53]:
import numpy as np
import math
import matplotlib.pyplot as plt
import sympy as sp
from sympy import Matrix

In [88]:
#defining material properties
E = 1
v = 0.3
t = 1
ro = 1

#Constitutive matrix for plain stress condition
C = (E/(1+v))*np.array([[1/(1-v), v/(1-v), 0], [v/(1-v), 1/(1-v), 0], [0, 0, 1/2]])

## Stiffness Matrix

In [55]:
#shape functions for linear Quad elements
eta = sp.Symbol("eta")
xi = sp.Symbol("xi")

n1 = (1-eta)*(1-xi)/4
n2 = (1-eta)*(1+xi)/4
n3 = (1+eta)*(1+xi)/4
n4 = (1+eta)*(1-xi)/4
n = Matrix([n1, n2, n3, n4])
#n

In [56]:
#N matrix
N = Matrix([[n1, 0, n2, 0, n3, 0, n4, 0], [0, n1, 0, n2, 0, n3, 0, n4]])
#N

In [57]:
#Jacobian Matrix
def jacobian(x, y):

    #mapping x-y to xi-eta coordinate system
    X = sp.Add(sp.Mul(n1,x[0]), sp.Mul(n2,x[1]), sp.Mul(n3,x[2]), sp.Mul(n4,x[3]))
    Y = sp.Add(sp.Mul(n1,y[0]), sp.Mul(n2,y[1]), sp.Mul(n3,y[2]), sp.Mul(n4,y[3]))

    j1 = sp.diff(X, xi)
    j2 = sp.diff(X, eta)
    j3 = sp.diff(Y, xi)
    j4 = sp.diff(Y, eta)

    J = Matrix([[j1, j2], [j3, j4]])

    return J

In [58]:
"""x = np.array([0,3,3.5,0.5])
y = np.array([0,1,3.2,3])
jacob = jacobian(x, y)
jacob"""

'x = np.array([0,3,3.5,0.5])\ny = np.array([0,1,3.2,3])\njacob = jacobian(x, y)\njacob'

In [59]:
#B1 matrix 
B_1 = Matrix([[1, 0, 0, 0],[0, 0, 0, 1],[0, 1, 1, 0]])

In [60]:
#coordinate mapping and B2 matrix
def B_2(x, y):

    J = jacobian(x, y)
    zero = Matrix([[0, 0], [0, 0]])
    J_inv_t = J.inv().T
    J1 = J_inv_t.row_join(zero)
    J2 = zero.row_join(J_inv_t)
    B2 = J1.col_join(J2)

    return (B2)

In [61]:
"""x = np.array([0,3,3.5,0.5])
y = np.array([0,1,3.2,3])
b2 = B_2(x, y)
b2"""

'x = np.array([0,3,3.5,0.5])\ny = np.array([0,1,3.2,3])\nb2 = B_2(x, y)\nb2'

In [62]:
#B3 Matrix
j = 0
B_3 = sp.zeros(4,8)
for i in range(0,8):
    if i%2==0:
        B_3[0,i] = sp.diff(n[j], xi)
        B_3[1,i] = sp.diff(n[j], eta)
        B_3[2,i+1] = sp.diff(n[j], xi)
        B_3[3,i+1] = sp.diff(n[j], eta)
        j = j + 1
#B_3

In [63]:
#Calculating B Matrix
def B_Matrix(x, y):
    B = B_1 * B_2(x, y) * B_3
    return B

In [64]:
"""#x-y coordinates of nodes in element
x = np.array([0,3,3.5,0.5])
y = np.array([0,1,3.2,3])
B_m = B_Matrix(x, y)
B_m"""

'#x-y coordinates of nodes in element\nx = np.array([0,3,3.5,0.5])\ny = np.array([0,1,3.2,3])\nB_m = B_Matrix(x, y)\nB_m'

In [65]:
#stiffness matrix using gaussian quadrature
def stiffness(x, y):
    B = B_Matrix(x, y)
    J = jacobian(x, y)
    k = B.T * C * B * J.det()
    K = sp.lambdify([xi, eta], k)
    r = math.sqrt(1/3)
    I = K(r, r) + K(r, -r) + K(-r, r) + K(-r, -r)
    
    return I

In [66]:
x = np.array([0,3,3.5,0.5])
y = np.array([0,1,3.2,3])
k_e = stiffness(x, y)

In [67]:
k_e

array([[ 3.22589738e-01,  1.00264451e-01, -2.53618645e-01,
         7.74655624e-05, -1.01186539e-01, -1.43610223e-01,
         3.22154463e-02,  4.32683061e-02],
       [ 1.00264451e-01,  3.88968901e-01,  2.75499930e-02,
         8.95588957e-02, -1.43610223e-01, -1.38009391e-01,
         1.57957787e-02, -3.40518406e-01],
       [-2.53618645e-01,  2.75499930e-02,  7.12013087e-01,
        -3.01881216e-01, -4.95334272e-02,  3.18003713e-02,
        -4.08861014e-01,  2.42530852e-01],
       [ 7.74655624e-05,  8.95588957e-02, -3.01881216e-01,
         7.83882080e-01,  5.92728988e-02, -4.33180379e-01,
         2.42530852e-01, -4.40260597e-01],
       [-1.01186539e-01, -1.43610223e-01, -4.95334272e-02,
         5.92728988e-02,  3.90713799e-01,  8.69272908e-02,
        -2.39993833e-01, -2.58996654e-03],
       [-1.43610223e-01, -1.38009391e-01,  3.18003713e-02,
        -4.33180379e-01,  8.69272908e-02,  4.66187212e-01,
         2.48825609e-02,  1.05002558e-01],
       [ 3.22154463e-02,  1.579577

## Boundary conditions

In [68]:
#BC1 - Concentrated loads
p = np.array([0,0,0,0,2,2,0,0])

#BC2 - body forces - gravity
b = np.array([0,-1])

#BC3 - Distributed loads
q = np.array([-1,0])

### Concentrated Forces

In [69]:
def f_conc(p):
    f1 = Matrix(p)
    return f1

### Body Forces

In [70]:
def f_body(b):
    J = jacobian(x, y)
    b_f = N.T * Matrix(b) * J.det()
    K = sp.lambdify([xi, eta], b_f)
    r = math.sqrt(1/3)
    I = K(r, r) + K(r, -r) + K(-r, r) + K(-r, -r)    
    return I

In [71]:
b = np.array([0,-1])
vol = f_body(b)
vol

array([[ 0.        ],
       [-1.95833333],
       [ 0.        ],
       [-1.75833333],
       [ 0.        ],
       [-1.79166667],
       [ 0.        ],
       [-1.99166667]])

### Distributed Forces

Still need some work here to solve dl_dita scalar value automatically

In [72]:
#need some work here in regards of automation

def f_dist(q):
    d_f = N.T * Matrix(q)
    K = sp.lambdify([xi, eta], d_f)
    I = 2*K(1,0)
    dl_dita = math.sqrt(509)/20

    return I*dl_dita

In [73]:
fs = f_dist(q)
fs

array([[-0.        ],
       [ 0.        ],
       [-1.12805142],
       [ 0.        ],
       [-1.12805142],
       [ 0.        ],
       [-0.        ],
       [ 0.        ]])

In [74]:
f_total = f_conc(p) + f_body(b) + f_dist(q)

In [75]:
f_total = np.array(f_total.tolist(), dtype=np.float64)
f_total

array([[ 0.        ],
       [-1.95833333],
       [-1.12805142],
       [-1.75833333],
       [ 0.87194858],
       [ 0.20833333],
       [ 0.        ],
       [-1.99166667]])

## Nodal Displacements

you must drop rows and columns for each element, where the nodes hav known values. So when to drop them before or after making global stiffness matrix.

In [99]:
#Removing rows and columns corresponding to known values of displacements
K_global = k_e[4: , 4: ]
f_global = f_total[4: ]

In [100]:
u = np.linalg.solve(K_global, f_global)

In [101]:
u

array([[ 1.18986424],
       [ 1.11203984],
       [-1.25401361],
       [-3.64099723]])

In [103]:
#inserting known values of displacements from BCs
disp = np.concatenate((np.zeros((4,1)),u))
disp

array([[ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 1.18986424],
       [ 1.11203984],
       [-1.25401361],
       [-3.64099723]])

## Reaction Forces

In [105]:
R_f = np.matmul(k_e, disp)
R_f
#This includes the externally applied force as well so to gett the actual reaction force we subtract the nodal forces

array([[-4.78036924e-01],
       [ 8.95669842e-01],
       [-3.93911659e-01],
       [ 8.87663491e-01],
       [ 8.71948583e-01],
       [ 2.08333333e-01],
       [ 2.22044605e-16],
       [-1.99166667e+00]])

In [106]:
Reaction_f = R_f - f_total
Reaction_f

array([[-4.78036924e-01],
       [ 2.85400318e+00],
       [ 7.34139759e-01],
       [ 2.64599682e+00],
       [-1.11022302e-16],
       [ 5.55111512e-17],
       [ 2.22044605e-16],
       [ 2.22044605e-16]])

## Displacement Field

In [107]:
Disp_f = N * disp

In [108]:
Disp_f

Matrix([
[ -0.31350340205979*(1 - xi)*(eta + 1) + 0.297466060769684*(eta + 1)*(xi + 1)],
[-0.910249307960854*(1 - xi)*(eta + 1) + 0.278009958848686*(eta + 1)*(xi + 1)]])

## Strain

In [111]:
strain = B_Matrix(x, y) * disp

In [112]:
strain

Matrix([
[                                                                                                                                                                                                                                                   -1.25401360823916*(1/4 - xi/4)*(0.2*eta - 0.3)/(0.05*eta - 0.3*xi + 1.875) - 1.25401360823916*(1.3 - 0.2*xi)*(-eta/4 - 1/4)/(0.05*eta - 0.3*xi + 1.875) + 1.18986424307874*(1.3 - 0.2*xi)*(eta/4 + 1/4)/(0.05*eta - 0.3*xi + 1.875) + 1.18986424307874*(0.2*eta - 0.3)*(xi/4 + 1/4)/(0.05*eta - 0.3*xi + 1.875)],
[                                                                                                                                                                                                                                                                                                               -5.46149584776512*(1/4 - xi/4)/(0.05*eta - 0.3*xi + 1.875) + 0.910249307960854*(-eta/4 - 1/4)/(0.05*eta - 0.3*xi + 1.875) - 0.27800995884

## Stress

In [115]:
stress = C * strain

In [116]:
stress

Matrix([
[  -1.37803693213094*(1/4 - xi/4)*(0.2*eta - 0.3)/(0.05*eta - 0.3*xi + 1.875) - 1.80049313662586*(1/4 - xi/4)/(0.05*eta - 0.3*xi + 1.875) - 1.37803693213094*(1.3 - 0.2*xi)*(-eta/4 - 1/4)/(0.05*eta - 0.3*xi + 1.875) + 1.30754312426235*(1.3 - 0.2*xi)*(eta/4 + 1/4)/(0.05*eta - 0.3*xi + 1.875) + 0.300082189437644*(-eta/4 - 1/4)/(0.05*eta - 0.3*xi + 1.875) + 1.30754312426235*(0.2*eta - 0.3)*(xi/4 + 1/4)/(0.05*eta - 0.3*xi + 1.875) - 0.0916516347852811*(eta/4 + 1/4)/(0.05*eta - 0.3*xi + 1.875) + 0.549909808711687*(xi/4 + 1/4)/(0.05*eta - 0.3*xi + 1.875)],
[  -0.413411079639283*(1/4 - xi/4)*(0.2*eta - 0.3)/(0.05*eta - 0.3*xi + 1.875) - 6.00164378875288*(1/4 - xi/4)/(0.05*eta - 0.3*xi + 1.875) - 0.413411079639283*(1.3 - 0.2*xi)*(-eta/4 - 1/4)/(0.05*eta - 0.3*xi + 1.875) + 0.392262937278704*(1.3 - 0.2*xi)*(eta/4 + 1/4)/(0.05*eta - 0.3*xi + 1.875) + 1.00027396479215*(-eta/4 - 1/4)/(0.05*eta - 0.3*xi + 1.875) + 0.392262937278704*(0.2*eta - 0.3)*(xi/4 + 1/4)/(0.05*eta - 0.3*xi + 1.875) - 

Scope for later versions
1) Instead of using one element, use four. Create a function to generate Global stiffness matrix and force value.
2) Incorporate point moment and distributed moment (might need to derive theoretically everything. Nodal dof will change in this case.)
3) Improving distributed loads (dl_dita matrix)
4) Visualizations